# Importing Packages

In [2]:
import pandas as pd
import sqlite3

In [3]:
conn=sqlite3.connect('mydatabase.db') #Creating Database
cursor=conn.cursor() #connection to the database storing in Cursor

# Creating Tables in DataBase and Storing Data in Tables

In [4]:
create_translation_table="""
CREATE TABLE translation(product_category_name TEXT PRIMARY KEY,
product_category_name_english TEXT)
"""
create_products_table="""
CREATE TABLE products(product_id TEXT PRIMARY KEY,
product_category_name TEXT,product_name_length INTEGER,
product_description_length INTEGER,product_photos_qty INTEGER,
product_weight_g INTEGER,product_length_cm INTEGER,
product_height_cm INTEGER,product_width_cm INTEGER,
FOREIGN KEY(product_category_name) REFERENCES translation(product_category_name))
"""
create_leads_table="""
CREATE TABLE leads(mql_id TEXT PRIMARY KEY,
first_contact_date TEXT,landing_page_id TEXT,origin TEXT)
"""
create_geolocation_table="""
CREATE TABLE geolocation(geolocation_zip_code_prefix INTEGER,
geolocation_lat NUMERIC,gelocation_lng NUMERIC,
geolocation_city TEXT,geolocation_state TEXT)
"""
create_sellers_table="""
CREATE TABLE sellers(seller_id TEXT PRIMARY KEY,seller_zip_code_prefix INTEGER,
seller_city TEXT,seller_state TEXT,
FOREIGN KEY(seller_zip_code_prefix) REFERENCES geolocation(geolocation_zip_code_prefix))
"""
create_customers_table="""
CREATE TABLE customers(customer_id TEXT PRIMARY KEY,customer_unique_id TEXT,
customer_zip_code_prefix INTEGER,customer_city TEXT,customer_state TEXT,
FOREIGN KEY(customer_zip_code_prefix) REFERENCES geolocation(geolocation_zip_code_prefix))
"""
create_orders_table="""
CREATE TABLE orders(order_id TEXT PRIMARY KEY,customer_id TEXT,
order_status TEXT,order_purchase_timestamp TEXT,order_approved_at TEXT,
order_delivered_carrier_date TEXT,order_delivered_customer_date TEXT,
order_estimated_delivery_date TEXT,FOREIGN KEY(customer_id) REFERENCES customers(customer_id))
"""
create_order_items_table="""
CREATE TABLE order_items(order_id TEXT,order_item_id TEXT,
product_id TEXT,seller_id TEXT,shipping_limit_date TEXT,
price NUMERIC,freight_value NUMERIC,FOREIGN KEY(order_id) REFERENCES orders(order_id),
FOREIGN KEY(product_id) REFERENCES products(product_id),FOREIGN KEY(seller_id) REFERENCES sellers(seller_id))
"""
create_order_payments="""
CREATE TABLE order_payments(order_id TEXT,payment_sequential INTEGER,
payment_type TEXT,payment_installments INTEGER,payment_value NUMERIC,
FOREIGN KEY(order_id) REFERENCES orders(order_id))
"""
create_order_reviews="""
CREATE TABLE order_reviews(review_id PRIMARY KEY,order_id TEXT,
review_score INTEGER,review_comment_title TEXT,review_comment_message TEXT,
review_creation_date TEXT,review_answer_timestamp TEXT,
FOREIGN KEY(order_id) REFERENCES orders(order_id))
"""
create_deals_table="""
CREATE TABLE deals(mql_id TEXT PRIMARY KEY,seller_id TEXT,
sdr_id TEXT,sr_id TEXT,won_date TEXT,bussiness_segment TEXT,
lead_type TEXT,lead_behaviour_profile TEXT,has_company TEXT,
average_stock TEXT,bussiness_type TEXT,declared_product_catlog_size NUMERIC,
declared_monthly_revenue NUMERIC,FOREIGN KEY(seller_id) REFERENCES sellers(seller_id))
"""

In [5]:
cursor.execute(create_translation_table)
df=pd.read_csv('product_category_name_translation.csv')
df.to_sql('translation',conn,if_exists='replace',index=False)

71

In [6]:
cursor.execute(create_products_table)
df=pd.read_csv('products_dataset.csv')
df.to_sql('products',conn,if_exists='replace',index=False)

32951

In [7]:
cursor.execute(create_leads_table)
df=pd.read_csv('marketing_qualified_leads_dataset.csv')
df.to_sql('leads',conn,if_exists='replace',index=False)

8000

In [8]:
cursor.execute(create_geolocation_table)
df=pd.read_csv('geolocation_dataset.csv')
df.to_sql('geolocation',conn,if_exists='replace',index=False)

1000163

In [9]:
cursor.execute(create_sellers_table)
df=pd.read_csv('sellers_dataset.csv')
df.to_sql('sellers',conn,if_exists='replace',index=False)

3095

In [10]:
cursor.execute(create_customers_table)
df=pd.read_csv('customers_dataset.csv')
df.to_sql('customers',conn,if_exists='replace',index=False)

99441

In [11]:
cursor.execute(create_orders_table)
df=pd.read_csv('orders_dataset.csv')
df.to_sql('orders',conn,if_exists='replace',index=False)

99441

In [12]:
cursor.execute(create_order_items_table)
df=pd.read_csv('order_items_dataset.csv')
df.to_sql('order_items',conn,if_exists='replace',index=False)

112650

In [13]:
cursor.execute(create_order_payments)
df=pd.read_csv('order_payments_dataset.csv')
df.to_sql('order_payments',conn,if_exists='replace',index=False)

103886

In [14]:
cursor.execute(create_order_reviews)
df=pd.read_csv('order_reviews_dataset.csv')
df.to_sql('order_reviews',conn,if_exists='replace',index=False)

100000

In [15]:
cursor.execute(create_deals_table)
df=pd.read_csv('closed_deals_dataset.csv')
df.to_sql('deals',conn,if_exists='replace',index=False)

842

# List of Tables in Database

In [16]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

translation
products
leads
geolocation
sellers
customers
orders
order_items
order_payments
order_reviews
deals


# Geographical Analysis
Count the number of customers in each city and state

In [17]:
cursor.execute("""SELECT g.geolocation_city AS city,g.geolocation_state AS state,COUNT(DISTINCT c.customer_id) AS num_customers
FROM customers c JOIN geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
GROUP BY city, state ORDER BY num_customers DESC
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,city,state,num_customers
0,sao paulo,SP,15586
1,são paulo,SP,15406
2,rio de janeiro,RJ,6923
3,belo horizonte,MG,2789
4,brasilia,DF,1951
...,...,...,...
6050,água fria de goiás,GO,1
6051,água preta,PE,1
6052,águas frias,SC,1
6053,érico cardoso,BA,1


# Customer Location Analysis
 Determine the top states with the highest number of customers

In [18]:
cursor.execute("""SELECT customer_state,COUNT(DISTINCT customer_id) AS num_customers 
FROM customers GROUP BY customer_state ORDER BY num_customers DESC LIMiT 5
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,customer_state,num_customers
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045


# Average Delivery Time
Calculate the average time it takes for orders to be delivered to customers

In [19]:
cursor.execute("""SELECT AVG(strftime('%j', order_delivered_customer_date) - strftime('%j', order_approved_at)) AS avg_delivery_time_days
FROM orders WHERE order_delivered_customer_date IS NOT NULL
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,avg_delivery_time_days
0,2.755562


# Delayed Deliveries
Orders that were delivered later than estimated date

In [20]:
cursor.execute("""SELECT o.order_id,o.order_estimated_delivery_date,o.order_delivered_customer_date 
FROM orders o WHERE order_delivered_customer_date > order_estimated_delivery_date
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,order_id,order_estimated_delivery_date,order_delivered_customer_date
0,203096f03d82e0dffbc41ebc2e2bcfb7,2017-09-28 00:00:00,2017-10-09 22:23:46
1,fbf9ac61453ac646ce8ad9783d7d0af6,2018-03-12 00:00:00,2018-03-21 22:03:54
2,8563039e855156e48fccee4d611a3196,2018-03-20 00:00:00,2018-03-20 00:59:25
3,6ea2f835b4556291ffdc53fa0b3b95e8,2017-12-21 00:00:00,2017-12-28 18:59:23
4,66e4624ae69e7dc89bd50222b59f581f,2018-04-02 00:00:00,2018-04-03 13:28:46
...,...,...,...
7822,b3112ca67f3afd4e20cf2ee91fc4f804,2018-08-16 00:00:00,2018-08-21 00:03:26
7823,0fa1fab1d7c1211c824596ed5e111e3c,2018-04-02 00:00:00,2018-04-05 19:59:49
7824,38e9133ce29f6bbe35aed9c3863dce01,2017-10-31 00:00:00,2017-11-21 17:06:59
7825,d692ef54145c9cb3322ec2e5508aa3f4,2018-04-09 00:00:00,2018-04-11 00:48:31


# Monthly Revenue
Calculate the monthly revenue over time

In [21]:
cursor.execute("""SELECT strftime('%Y-%m', o.order_purchase_timestamp) AS month,SUM(oi.price + oi.freight_value) AS total_revenue
FROM orders o JOIN order_items oi ON o.order_id = oi.order_id GROUP BY month ORDER BY month
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,month,total_revenue
0,2016-09,354.75
1,2016-10,56808.84
2,2016-12,19.62
3,2017-01,137188.49
4,2017-02,286280.62
5,2017-03,432048.59
6,2017-04,412422.24
7,2017-05,586190.95
8,2017-06,502963.04
9,2017-07,584971.62


# Revenue Distribution Business Segement
Determine the distribution of revenue among different business segments.

In [5]:
cursor.execute("""SELECT t.product_category_name_english,SUM(oi.price + oi.freight_value) AS total_revenue
FROM deals d JOIN order_items oi ON d.seller_id = oi.seller_id JOIN products p ON oi.product_id=p.product_id
JOIN translation t ON p.product_category_name=t.product_category_name GROUP BY t.product_category_name_english
ORDER BY total_revenue DESC;
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,product_category_name_english,total_revenue
0,health_beauty,123156.00
1,watches_gifts,112043.40
2,housewares,76197.89
3,pet_shop,40314.72
4,sports_leisure,35464.21
5,baby,30881.91
6,auto,30027.05
7,small_appliances_home_oven_and_coffee,27467.47
8,telephony,26562.61
9,bed_bath_table,24073.10


# Seller Revenue Ranking
Ranking Sellers by their total revenue

In [22]:
cursor.execute("""SELECT s.seller_id,SUM(oi.price + oi.freight_value) AS total_revenue
FROM sellers s JOIN order_items oi ON s.seller_id = oi.seller_id GROUP BY s.seller_id
ORDER BY total_revenue DESC LIMIT 10
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,seller_id,total_revenue
0,4869f7a5dfa277a7dca6462dcf3b52b2,249640.70
1,7c67e1448b00f6e969d365cea6b010ab,239536.44
2,53243585a1d6dc2643021fd1853d8905,235856.68
3,4a3ca9315b744ce9f8e9374361493884,235539.96
4,fa1c13f2614d7b5c4749cbc52fecda94,204084.73
5,da8622b14eb17ae2831f4ac5b9dab84a,185192.32
6,7e93a43ef30c4f03f38b393420bc753a,182754.05
7,1025f0e2d44d7041d6cf58b6550e0bfa,172860.69
8,7a67c85e85bb2ce8582c35f2203ad736,162648.38
9,955fee9216a65b617aa5c0531780ce60,160602.68


# Seller Performance Trend
Calculate the average monthly revenue for each seller

In [23]:
cursor.execute("""SELECT s.seller_id,strftime('%Y-%m', o.order_purchase_timestamp) AS month,
AVG(oi.price + oi.freight_value) AS avg_monthly_revenue FROM sellers s
JOIN order_items oi ON s.seller_id = oi.seller_id JOIN orders o ON oi.order_id = o.order_id
GROUP BY s.seller_id, month ORDER BY s.seller_id, month
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,seller_id,month,avg_monthly_revenue
0,0015a82c2db000af6aaaf3ae2ecb0532,2017-09,916.020000
1,0015a82c2db000af6aaaf3ae2ecb0532,2017-10,916.020000
2,001cca7ae9ae17fb1caed9dfb1094831,2017-02,117.763636
3,001cca7ae9ae17fb1caed9dfb1094831,2017-03,124.941176
4,001cca7ae9ae17fb1caed9dfb1094831,2017-04,140.201250
...,...,...,...
16436,fffd5413c0700ac820c7069d66d98c89,2018-08,156.082500
16437,ffff564a4f9085cd26170f4732393726,2016-10,73.485000
16438,ffff564a4f9085cd26170f4732393726,2017-01,54.563333
16439,ffff564a4f9085cd26170f4732393726,2017-03,106.658182


# Seller Performance by State
Calculate the average review score for each seller by state

In [24]:
cursor.execute("""SELECT s.seller_id,g.geolocation_state AS state,AVG(orv.review_score) AS avg_review_score
FROM sellers s JOIN order_items oi ON s.seller_id = oi.seller_id JOIN order_reviews orv ON oi.order_id = orv.order_id
JOIN orders o ON oi.order_id = o.order_id JOIN customers c ON o.customer_id = c.customer_id
JOIN geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix GROUP BY s.seller_id, state
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,seller_id,state,avg_review_score
0,0015a82c2db000af6aaaf3ae2ecb0532,MG,4.554348
1,0015a82c2db000af6aaaf3ae2ecb0532,PR,5.000000
2,001cca7ae9ae17fb1caed9dfb1094831,AC,5.000000
3,001cca7ae9ae17fb1caed9dfb1094831,BA,3.523788
4,001cca7ae9ae17fb1caed9dfb1094831,CE,3.842105
...,...,...,...
17556,ffff564a4f9085cd26170f4732393726,RJ,1.000000
17557,ffff564a4f9085cd26170f4732393726,RS,1.000000
17558,ffff564a4f9085cd26170f4732393726,SC,1.000000
17559,ffff564a4f9085cd26170f4732393726,SP,3.474725


# Busiest Days of the Week
Determining the days of week with highest number of orders

In [25]:
cursor.execute("""SELECT strftime('%w', order_purchase_timestamp) AS day_of_week,COUNT(DISTINCT order_id) AS num_orders
FROM orders GROUP BY day_of_week ORDER BY num_orders DESC
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,day_of_week,num_orders
0,1,16196
1,2,15963
2,3,15552
3,4,14761
4,5,14122
5,0,11960
6,6,10887


# Popular Product Categories
Find the most popular product categories based on the number of orders

In [26]:
cursor.execute("""SELECT t.product_category_name_english,COUNT(DISTINCT o.order_id) AS num_orders
FROM products p JOIN order_items oi ON p.product_id = oi.product_id JOIN orders o ON oi.order_id = o.order_id
JOIN translation t on p.product_category_name=t.product_category_name
GROUP BY t.product_category_name_english ORDER BY num_orders DESC LIMIT 10
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,product_category_name_english,num_orders
0,bed_bath_table,9417
1,health_beauty,8836
2,sports_leisure,7720
3,computers_accessories,6689
4,furniture_decor,6449
5,housewares,5884
6,watches_gifts,5624
7,telephony,4199
8,auto,3897
9,toys,3886


# Average Review Score by Product
Calculate the average review score for each product

In [27]:
cursor.execute("""SELECT t.product_category_name_english,AVG(orv.review_score) AS avg_review_score
FROM products p LEFT JOIN order_items oi ON p.product_id = oi.product_id
LEFT JOIN order_reviews orv ON oi.order_id = orv.order_id 
JOIN translation t ON p.product_category_name=t.product_category_name GROUP BY t.product_category_name_english
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,product_category_name_english,avg_review_score
0,agro_industry_and_commerce,4.000000
1,air_conditioning,3.919192
2,art,3.918660
3,arts_and_craftmanship,4.125000
4,audio,3.808219
...,...,...
66,stationery,4.181386
67,tablets_printing_image,4.048193
68,telephony,3.935604
69,toys,4.138667


# Repeat Customer Rate
Calculate the percentage of cutomers who have more than one order

In [28]:
cursor.execute("""SELECT COUNT(DISTINCT CASE WHEN order_count > 1 THEN customer_id END) AS repeat_customers,
COUNT(DISTINCT customer_id) AS total_customers,
ROUND(COUNT(DISTINCT CASE WHEN order_count > 1 THEN customer_id END) * 100.0 / COUNT(DISTINCT customer_id), 2) AS repeat_customer_rate
FROM (SELECT customer_id,COUNT(DISTINCT order_id) AS order_count FROM orders
GROUP BY customer_id) customer_orders
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,repeat_customers,total_customers,repeat_customer_rate
0,0,99441,0.0


# Deal Success Rate
Calculate the success rate of won deals for different lead behaviors

In [29]:
cursor.execute("""SELECT d.lead_behaviour_profile,
COUNT(DISTINCT CASE WHEN d.won_date IS NOT NULL THEN d.mql_id END) AS converted_deals,
COUNT(DISTINCT d.mql_id) AS total_deals,
ROUND(COUNT(DISTINCT CASE WHEN d.won_date IS NOT NULL THEN d.mql_id END) * 100.0 / COUNT(DISTINCT d.mql_id), 2) AS success_rate
FROM deals d GROUP BY d.lead_behaviour_profile
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,lead_behaviour_profile,converted_deals,total_deals,success_rate
0,None,177,177,100.0
1,cat,407,407,100.0
2,"cat, wolf",8,8,100.0
3,eagle,123,123,100.0
4,"eagle, cat",3,3,100.0
5,"eagle, wolf",3,3,100.0
6,shark,24,24,100.0
7,"shark, cat",1,1,100.0
8,"shark, wolf",1,1,100.0
9,wolf,95,95,100.0


# Growth Rate by Month
Calculate growth rate of orders month by month

In [30]:
cursor.execute("""SELECT strftime('%Y-%m', order_purchase_timestamp) AS order_month,
COUNT(order_id) AS num_orders,
LAG(COUNT(order_id)) OVER (ORDER BY strftime('%Y-%m', order_purchase_timestamp)) AS prev_month_orders,
CASE
    WHEN LAG(COUNT(order_id)) OVER (ORDER BY strftime('%Y-%m', order_purchase_timestamp)) = 0 THEN 0
    ELSE ROUND((COUNT(order_id) - LAG(COUNT(order_id)) OVER 
    (ORDER BY strftime('%Y-%m', order_purchase_timestamp))) * 100.0 / LAG(COUNT(order_id)) OVER (ORDER BY strftime('%Y-%m', order_purchase_timestamp)), 2)
END AS growth_rate
FROM orders GROUP BY order_month ORDER BY order_month
""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,order_month,num_orders,prev_month_orders,growth_rate
0,2016-09,4,NaN,NaN
1,2016-10,324,4.0,8000.00
2,2016-12,1,324.0,-99.69
3,2017-01,800,1.0,79900.00
4,2017-02,1780,800.0,122.50
5,2017-03,2682,1780.0,50.67
6,2017-04,2404,2682.0,-10.37
7,2017-05,3700,2404.0,53.91
8,2017-06,3245,3700.0,-12.30
9,2017-07,4026,3245.0,24.07


# Sellers with first orders
Query shows the no.of sellers who already have their first orders

In [3]:
cursor.execute("""SELECT count(seller_id) AS num_sellers
             FROM deals
             WHERE EXISTS (SELECT seller_id
                           FROM order_items
                           WHERE deals.seller_id=order_items.seller_id)""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,num_sellers
0,380


# Average order value
Shows the average order value of each seller

In [4]:
cursor.execute("""SELECT t1.seller_id, business_segment,ROUND(total_order_value/num_orders*100,2) AS avg_order_value
                  FROM (SELECT seller_id,COUNT(DISTINCT order_id) AS num_orders,SUM(price) + SUM(freight_value)AS total_order_value
                  FROM order_items GROUP BY seller_id) t1 JOIN deals d ON t1.seller_id = d.seller_id
                  ORDER BY avg_order_value DESC LIMIT 10""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,seller_id,business_segment,avg_order_value
0,1444c08e64d55fb3c25f0f09c07ffcf2,car_accessories,281874.00
1,c004e5ea15737026cecaee0447e00b75,construction_tools_house_garden,243716.00
2,8de8fe3af4449ed695d2434c933ed73e,air_conditioning,215535.00
3,d7827b2af99326a03b0ed9c7a24db0d3,construction_tools_house_garden,155670.00
4,04843805947f0fc584fc1969b6e50fe7,home_decor,147476.00
5,9b1585752613ec342d03bbab9997ec48,car_accessories,144968.00
6,33dd941c27854f7625b968cc6195a552,household_utilities,143368.20
7,0873d9f8f36123f8d910f4760e788cfb,audio_video_electronics,123777.50
8,ba90964cff9b9e0e6f32b23b82465f7b,small_appliances,119798.68
9,28872dc528e978a639754bc8c2ce5a4c,household_utilities,103890.00


# Different payment types
Count of each payment type used by customers

In [5]:
cursor.execute("""SELECT payment_type,
                  COUNT(order_id) AS num_payments
                  FROM order_payments
                  GROUP BY payment_type
                  ORDER BY num_payments DESC""")
data=pd.DataFrame(cursor.fetchall())
data.columns=[x[0] for x in cursor.description]
data

,payment_type,num_payments
0,credit_card,76795
1,boleto,19784
2,voucher,5775
3,debit_card,1529
4,not_defined,3
